In [4]:
# start tensorflow interactiveSession                baseline           F1 = 0.7377938517
import tensorflow as tf
import pandas as pd
import numpy as np
from PIL import Image

sess = tf.InteractiveSession()

#####################################################     Net Define     ##################################################### 

# weight initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

# convolution
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
# pooling
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Create the model
# placeholder
x = tf.placeholder("float", [None, 2500])
y_ = tf.placeholder("float", [None, 3])


# first convolutinal layer
w_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 50, 50, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, w_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# second convolutional layer
w_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# densely connected layer
w_fc1 = weight_variable([13*13*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 13*13*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

# dropout
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# readout layer
w_fc2 = weight_variable([1024, 3])
b_fc2 = bias_variable([3])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

# train and evaluate the model
#交叉熵作为损失函数
delta = 1e-7
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv+delta))
train_step = tf.train.GradientDescentOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.initialize_all_variables())


#####################################################       Train     ##################################################### 

image_path = 'E:/Age Detection of Indian Actors/train_DETg9GD/Train_Gray/'
data_path = 'E:/Age Detection of Indian Actors/train_DETg9GD/Train.csv'

data = pd.read_csv(data_path)
data = pd.get_dummies(data,columns = ['Class'])


def get_batch(data,batch_rate,seed):
    """
    本函数实现从训练集中随机抽取部分图像，格式适合输入tensorflow，同时得到样本label的独热编码
    input:
        batch_rate:    batch占训练集数据的比例
        data:          DataFrame       train.csv
        seed:          随机种子
    output:
        train_x:        训练图像tensor
        train_y:        训练图像one_hot编码的label
    """
    sample = data.sample(frac = batch_rate, random_state = seed)
    sample.index = list(range(0,sample.shape[0]))
    one_hot_sample = sample.drop(['ID'], axis = 1)
    train_y = one_hot_sample.values
    train_x = []
    for i in range(0,sample.shape[0]):
        image = image_path + sample.ID[i]
        im = np.array(Image.open(image))/255.0
        im = im.flatten()
        train_x.append(im)
    train_x = np.array(train_x)
    
    return train_x,train_y
    
    
    
    
    
batch_rate = 0.005
for i in range(50000):
    batch_x,batch_y = get_batch(data,batch_rate,i)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch_x, y_:batch_y, keep_prob:1.0})
        print ("step %d, train accuracy %g" %(i, train_accuracy))
    train_step.run(feed_dict={x:batch_x, y_:batch_y, keep_prob:0.5})

#print ("test accuracy %g" % accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0}))

E:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\client\session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


step 0, train accuracy 0.56
step 100, train accuracy 0.52
step 200, train accuracy 0.67
step 300, train accuracy 0.6
step 400, train accuracy 0.68
step 500, train accuracy 0.65
step 600, train accuracy 0.65
step 700, train accuracy 0.66
step 800, train accuracy 0.66
step 900, train accuracy 0.6
step 1000, train accuracy 0.6
step 1100, train accuracy 0.61
step 1200, train accuracy 0.65
step 1300, train accuracy 0.65
step 1400, train accuracy 0.61
step 1500, train accuracy 0.73
step 1600, train accuracy 0.73
step 1700, train accuracy 0.57
step 1800, train accuracy 0.68
step 1900, train accuracy 0.71
step 2000, train accuracy 0.63
step 2100, train accuracy 0.62
step 2200, train accuracy 0.62
step 2300, train accuracy 0.72
step 2400, train accuracy 0.57
step 2500, train accuracy 0.63
step 2600, train accuracy 0.63
step 2700, train accuracy 0.66
step 2800, train accuracy 0.63
step 2900, train accuracy 0.7
step 3000, train accuracy 0.68
step 3100, train accuracy 0.57
step 3200, train accurac

step 26100, train accuracy 0.84
step 26200, train accuracy 0.85
step 26300, train accuracy 0.82
step 26400, train accuracy 0.76
step 26500, train accuracy 0.81
step 26600, train accuracy 0.79
step 26700, train accuracy 0.87
step 26800, train accuracy 0.84
step 26900, train accuracy 0.84
step 27000, train accuracy 0.81
step 27100, train accuracy 0.8
step 27200, train accuracy 0.84
step 27300, train accuracy 0.76
step 27400, train accuracy 0.78
step 27500, train accuracy 0.73
step 27600, train accuracy 0.79
step 27700, train accuracy 0.79
step 27800, train accuracy 0.78
step 27900, train accuracy 0.83
step 28000, train accuracy 0.77
step 28100, train accuracy 0.76
step 28200, train accuracy 0.85
step 28300, train accuracy 0.79
step 28400, train accuracy 0.82
step 28500, train accuracy 0.83
step 28600, train accuracy 0.72
step 28700, train accuracy 0.76
step 28800, train accuracy 0.81
step 28900, train accuracy 0.81
step 29000, train accuracy 0.78
step 29100, train accuracy 0.8
step 29200

In [13]:
pred = tf.argmax(y_conv, 1)
P = pred.eval(feed_dict={x:batch_x, y_:batch_y, keep_prob:1.0})
P

array([0, 2, 2, 2, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 2, 2, 0, 1, 0, 2, 2,
       0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 2, 1, 2, 0, 1, 2, 0, 0, 0, 0, 0, 2, 2,
       2, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 2, 0,
       2, 0, 2, 0, 0, 2, 0, 2, 0, 2, 1, 2, 0, 2, 0, 1, 0, 2, 2, 0, 1, 0, 2,
       2, 2, 0, 0, 2, 0, 2, 0], dtype=int64)

In [9]:
image_path = 'E:/Age Detection of Indian Actors/test_Bh8pGW3/Test_Gray/'

test = pd.read_csv('E:/Age Detection of Indian Actors/test_Bh8pGW3/test.csv')

test_x = []
for i in range(0,test.shape[0]):
    image = image_path + test.ID[i]
    im = np.array(Image.open(image))/255.0
    im = im.flatten()
    test_x.append(im)
test_x = np.array(test_x)


ResourceExhaustedError: OOM when allocating tensor with shape[6636,32,50,50] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: Conv2D_6 = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Reshape_6, Variable_24/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: ArgMax_9/_37 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_70_ArgMax_9", tensor_type=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for curren

In [ ]:
P.shape

In [12]:
test_y = np.zeros((test.shape[0],3))
P = pred.eval(feed_dict={x:test_x, y_:test_y, keep_prob:1.0})

ResourceExhaustedError: OOM when allocating tensor with shape[6636,32,50,50] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: Conv2D_6 = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Reshape_6, Variable_24/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: ArgMax_9/_37 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_70_ArgMax_9", tensor_type=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for curren

In [24]:
test_x_0 = test_x[0:1500]
test_y_0 = test_y[0:1500]
P_0 = pred.eval(feed_dict={x:test_x_0, y_:test_y_0, keep_prob:1.0})

test_x_1 = test_x[1500:3000]
test_y_1 = test_y[1500:3000]
P_1 = pred.eval(feed_dict={x:test_x_1, y_:test_y_1, keep_prob:1.0})

test_x_2 = test_x[3000:4500]
test_y_2 = test_y[3000:4500]
P_2 = pred.eval(feed_dict={x:test_x_2, y_:test_y_2, keep_prob:1.0})

test_x_3 = test_x[4500:]
test_y_3 = test_y[4500:]
P_3 = pred.eval(feed_dict={x:test_x_3, y_:test_y_3, keep_prob:3.0})

In [28]:
P = np.hstack([P_0,P_1,P_2,P_3])
P.shape

(6636,)

In [29]:
test['pred'] = P

In [30]:
test.head()

,ID,pred
0,25321.jpg,0
1,989.jpg,2
2,19277.jpg,0
3,13093.jpg,0
4,5367.jpg,1


In [31]:
Dict = {0:'MIDDLE',1:'OLD', 2:'YOUNG'}
Class = []
for i in range(0,test.shape[0]):
    Class.append(Dict.get(test['pred'][i]))
    
pred = pd.DataFrame({'Class':Class,'ID':test.ID})
pred.head()

,Class,ID
0,MIDDLE,25321.jpg
1,YOUNG,989.jpg
2,MIDDLE,19277.jpg
3,MIDDLE,13093.jpg
4,OLD,5367.jpg


In [32]:
pred.to_csv('50_gray_pix_pred.csv', index = False)

In [1]:
# start tensorflow interactiveSession                baseline           F1 = 0.7606992164
import tensorflow as tf
import pandas as pd
import numpy as np
from PIL import Image

sess = tf.InteractiveSession()

#####################################################     Net Define     ##################################################### 

# weight initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

# convolution
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
# pooling
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Create the model
# placeholder
x = tf.placeholder("float", [None, 2500])
y_ = tf.placeholder("float", [None, 3])


# first convolutinal layer
w_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 50, 50, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, w_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# second convolutional layer
w_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# densely connected layer
w_fc1 = weight_variable([13*13*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 13*13*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

# dropout
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# readout layer
w_fc2 = weight_variable([1024, 3])
b_fc2 = bias_variable([3])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

# train and evaluate the model
#交叉熵作为损失函数
delta = 1e-7
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv+delta))
train_step = tf.train.GradientDescentOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.initialize_all_variables())


#####################################################       Train     ##################################################### 

image_path = 'E:/Age Detection of Indian Actors/train_DETg9GD/Train_Gray/'
data_path = 'E:/Age Detection of Indian Actors/train_DETg9GD/Train.csv'

data = pd.read_csv(data_path)
data = pd.get_dummies(data,columns = ['Class'])


def get_batch(data,batch_rate,seed):
    """
    本函数实现从训练集中随机抽取部分图像，格式适合输入tensorflow，同时得到样本label的独热编码
    input:
        batch_rate:    batch占训练集数据的比例
        data:          DataFrame       train.csv
        seed:          随机种子
    output:
        train_x:        训练图像tensor
        train_y:        训练图像one_hot编码的label
    """
    sample = data.sample(frac = batch_rate, random_state = seed)
    sample.index = list(range(0,sample.shape[0]))
    one_hot_sample = sample.drop(['ID'], axis = 1)
    train_y = one_hot_sample.values
    train_x = []
    for i in range(0,sample.shape[0]):
        image = image_path + sample.ID[i]
        im = np.array(Image.open(image))/255.0
        im = im.flatten()
        train_x.append(im)
    train_x = np.array(train_x)
    
    return train_x,train_y
    
    
    
    
    
batch_rate = 0.005
for i in range(64000):
    batch_x,batch_y = get_batch(data,batch_rate,i)
    if i%200 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch_x, y_:batch_y, keep_prob:1.0})
        print ("step %d, train accuracy %g" %(i, train_accuracy))
    train_step.run(feed_dict={x:batch_x, y_:batch_y, keep_prob:0.5})

#print ("test accuracy %g" % accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0, train accuracy 0.34
step 200, train accuracy 0.67
step 400, train accuracy 0.67
step 600, train accuracy 0.71
step 800, train accuracy 0.68
step 1000, train accuracy 0.61
step 1200, train accuracy 0.66
step 1400, train accuracy 0.58
step 1600, train accuracy 0.73
step 1800, train accuracy 0.66
step 2000, train accuracy 0.59
step 2200, train accuracy 0.65
step 2400, train accuracy 0.6
step 2600, train accuracy 0.7
step 2800, train accuracy 0.64
step 3000, train accuracy 0.68
step 3200, train accuracy 0.68
step 3400, train accuracy 0.73
step 3600, train accuracy 0.62
step 3800, train accuracy 0.73
step 4000, train accuracy 0.71
step 4200, train accuracy 0.57
step 4400, train accuracy 0.71
step 4600, train accuracy 0.73
step 4800, train accuracy 0.72
step 5000, train accuracy 0.72
step 5200, train accuracy 0.65
step 5400, train accuracy 0.66
step 5600, train accuracy 0.73
step 5800, train accuracy 0.66
step 

step 49800, train accuracy 0.93
step 50000, train accuracy 0.95
step 50200, train accuracy 0.97
step 50400, train accuracy 0.99
step 50600, train accuracy 0.99
step 50800, train accuracy 0.98
step 51000, train accuracy 0.97
step 51200, train accuracy 0.98
step 51400, train accuracy 0.97
step 51600, train accuracy 0.98
step 51800, train accuracy 0.96
step 52000, train accuracy 0.94
step 52200, train accuracy 0.99
step 52400, train accuracy 0.96
step 52600, train accuracy 0.98
step 52800, train accuracy 0.95
step 53000, train accuracy 0.94
step 53200, train accuracy 0.97
step 53400, train accuracy 0.95
step 53600, train accuracy 0.99
step 53800, train accuracy 0.99
step 54000, train accuracy 0.99
step 54200, train accuracy 0.95
step 54400, train accuracy 0.97
step 54600, train accuracy 0.93
step 54800, train accuracy 0.97
step 55000, train accuracy 0.97
step 55200, train accuracy 0.96
step 55400, train accuracy 0.98
step 55600, train accuracy 1
step 55800, train accuracy 0.98
step 56000,

In [3]:
image_path = 'E:/Age Detection of Indian Actors/test_Bh8pGW3/Test_Gray/'

test = pd.read_csv('E:/Age Detection of Indian Actors/test_Bh8pGW3/test.csv')

test_x = []
for i in range(0,test.shape[0]):
    image = image_path + test.ID[i]
    im = np.array(Image.open(image))/255.0
    im = im.flatten()
    test_x.append(im)
test_x = np.array(test_x)
test_y = np.zeros((test.shape[0],3))

pred = tf.argmax(y_conv, 1)

test_x_0 = test_x[0:1500]
test_y_0 = test_y[0:1500]
P_0 = pred.eval(feed_dict={x:test_x_0, y_:test_y_0, keep_prob:1.0})

test_x_1 = test_x[1500:3000]
test_y_1 = test_y[1500:3000]
P_1 = pred.eval(feed_dict={x:test_x_1, y_:test_y_1, keep_prob:1.0})

test_x_2 = test_x[3000:4500]
test_y_2 = test_y[3000:4500]
P_2 = pred.eval(feed_dict={x:test_x_2, y_:test_y_2, keep_prob:1.0})

test_x_3 = test_x[4500:]
test_y_3 = test_y[4500:]
P_3 = pred.eval(feed_dict={x:test_x_3, y_:test_y_3, keep_prob:3.0})

P = np.hstack([P_0,P_1,P_2,P_3])

test['pred'] = P

Dict = {0:'MIDDLE',1:'OLD', 2:'YOUNG'}
Class = []
for i in range(0,test.shape[0]):
    Class.append(Dict.get(test['pred'][i]))
    
pred = pd.DataFrame({'Class':Class,'ID':test.ID})
pred.head()

pred.to_csv('50_gray_pix_pred_64k_batch.csv', index = False)